# Preparación

In [ ]:
pip install --upgrade scikit-learn

In [ ]:
pip install scikit-tda

In [ ]:
pip install gudhi

In [ ]:
import time
import warnings
from ripser import ripser
import numpy as np
import matplotlib.pyplot as plt
from persim import plot_diagrams
import persim
import glob
from numpy import asarray
from PIL import Image
import os
import csv

from sklearn.utils import shuffle
from sklearn import cluster, datasets
from sklearn.preprocessing import StandardScaler
from itertools import cycle, islice
import random
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler

from sklearn.semi_supervised import LabelPropagation
from sklearn.semi_supervised import LabelSpreading
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.svm import SVC
import warnings

import pandas as pd

warnings.filterwarnings('ignore')

In [ ]:
!wget https://unirioja-my.sharepoint.com/:x:/g/personal/adines_unirioja_es/EQD3G4utUGFCpToyay-HSXMBAkYdarxgLB3kDpDDXLNPiA?download=1 -O primaIndianDiabetes.csv

In [ ]:
def load_prima():
  df=pd.read_csv("primaIndianDiabetes.csv", sep=',',header=None)
  df1 = df.iloc[:,:-1]
  df2=df.iloc[:,-1]
  return df1.values,df2.values

In [ ]:
def preparar_dataset(data,target,n_labeled=15, fold=0):
  numFolds=5
  scaler = StandardScaler()
  data=scaler.fit_transform(data)
  unos=list(np.where(target==1)[0])
  unos=shuffle(unos,random_state=10)

  ceros=list(np.where(target==0)[0])
  ceros=shuffle(ceros,random_state=10)

  k_ceros=len(ceros)//numFolds
  k_unos=len(unos)//numFolds

  fold_ceros=[ceros[k_ceros*i:k_ceros*(i+1)] for i in range(numFolds-1)]
  fold_unos=[unos[k_unos*i:k_unos*(i+1)] for i in range(numFolds-1)]

  fold_ceros.append(ceros[k_ceros*(numFolds-1):])
  fold_unos.append(unos[k_unos*(numFolds-1):])

  valid_ceros=fold_ceros[fold]
  valid_unos=fold_unos[fold]

  label_fold_ceros=fold_ceros[(fold+1)%numFolds][:n_labeled]
  label_fold_unos=fold_unos[(fold+1)%numFolds][:n_labeled]

  unlabeled_ceros=set()
  unlabeled_unos=set()

  for i in range(numFolds):
    if i!=fold and i!=(fold+1)%numFolds:
      unlabeled_ceros=unlabeled_ceros.union(set(fold_ceros[i]))
      unlabeled_unos=unlabeled_unos.union(set(fold_unos[i]))
    elif i==(fold+1)%numFolds:
      unlabeled_ceros=unlabeled_ceros.union(set(fold_ceros[i][n_labeled:]))
      unlabeled_unos=unlabeled_unos.union(set(fold_unos[i][n_labeled:]))


  puntos_ceros_valid=np.array(data)[valid_ceros]
  puntos_unos_valid=np.array(data)[valid_unos]

  puntos_ceros=np.array(data)[label_fold_ceros]
  puntos_unos=np.array(data)[label_fold_unos]

  X_unlabeled_ceros=np.array(data)[list(unlabeled_ceros)]
  X_unlabeled_unos=np.array(data)[list(unlabeled_unos)]


  print("SUMMARY")
  print("Train 0: "+str(len(puntos_ceros)))
  print("Train 1: "+str(len(puntos_unos)))
  print("Valid 0: "+str(len(puntos_ceros_valid)))
  print("Valid 1: "+str(len(puntos_unos_valid)))
  print("Unlabelled 0: "+str(len(X_unlabeled_ceros)))
  print("Unlabelled 1: "+str(len(X_unlabeled_unos)))

  return (puntos_ceros,puntos_unos,X_unlabeled_ceros,X_unlabeled_unos,puntos_ceros_valid,puntos_unos_valid)

In [ ]:
def analizar_punto(punto,puntos_ceros,puntos_unos,dgms_ceros,dgms_unos,th=0,dim=0):
  dgms_ceros_mod = ripser(np.array([punto.tolist()]+puntos_ceros.tolist()))['dgms'][dim]
  distance_bottleneck_cero = persim.bottleneck(dgms_ceros_mod, dgms_ceros, matching=False)

  dgms_unos_mod = ripser(np.array([punto.tolist()]+puntos_unos.tolist()))['dgms'][dim]
  distance_bottleneck_uno = persim.bottleneck(dgms_unos_mod, dgms_unos, matching=False)
  clase=-1

  if distance_bottleneck_cero>distance_bottleneck_uno:
    clase=1
  else:
    clase=0
  return clase

In [ ]:
def analizar_punto_th(punto,puntos_ceros,puntos_unos,dgms_ceros,dgms_unos,th=0.8,dim=0):
  dgms_ceros_mod = ripser(np.array([punto.tolist()]+puntos_ceros.tolist()))['dgms'][dim]
  distance_bottleneck_cero = persim.bottleneck(dgms_ceros_mod, dgms_ceros, matching=False)

  dgms_unos_mod = ripser(np.array([punto.tolist()]+puntos_unos.tolist()))['dgms'][dim]
  distance_bottleneck_uno = persim.bottleneck(dgms_unos_mod, dgms_unos, matching=False)

  distancia=distance_bottleneck_cero+distance_bottleneck_uno

  if max(distance_bottleneck_cero/distancia,distance_bottleneck_uno/distancia)<th:
    clase=-1
  elif distance_bottleneck_cero>distance_bottleneck_uno:
    clase=1
  else:
    clase=0
  return clase

In [ ]:
def analizar_punto_Wasserstein(punto,puntos_ceros,puntos_unos,dgms_ceros,dgms_unos,th=0,dim=0):
  dgms_ceros_mod = ripser(np.array([punto.tolist()]+puntos_ceros.tolist()))['dgms'][dim]
  distance_bottleneck_cero = persim.wasserstein(dgms_ceros_mod, dgms_ceros, matching=False)

  dgms_unos_mod = ripser(np.array([punto.tolist()]+puntos_unos.tolist()))['dgms'][dim]
  distance_bottleneck_uno = persim.wasserstein(dgms_unos_mod, dgms_unos, matching=False)
  clase=-1

  if distance_bottleneck_cero>distance_bottleneck_uno:
    clase=1
  else:
    clase=0
  return clase

In [ ]:
def analizar_punto_Wasserstein_th(punto,puntos_ceros,puntos_unos,dgms_ceros,dgms_unos,th=0.8,dim=0):
  dgms_ceros_mod = ripser(np.array([punto.tolist()]+puntos_ceros.tolist()))['dgms'][dim]
  distance_bottleneck_cero = persim.wasserstein(dgms_ceros_mod, dgms_ceros, matching=False)

  dgms_unos_mod = ripser(np.array([punto.tolist()]+puntos_unos.tolist()))['dgms'][dim]
  distance_bottleneck_uno = persim.wasserstein(dgms_unos_mod, dgms_unos, matching=False)

  distancia=distance_bottleneck_cero+distance_bottleneck_uno

  if max(distance_bottleneck_cero/distancia,distance_bottleneck_uno/distancia)<th:
    clase=-1
  elif distance_bottleneck_cero>distance_bottleneck_uno:
    clase=1
  else:
    clase=0
  return clase

In [ ]:
def analizar_puntos(data,target,func,th=0,reduccion=False,dim=0,fold=0):
  (puntos_ceros,puntos_unos,X_unlabeled_ceros,X_unlabeled_unos,X_unlabeled_ceros_valid,X_unlabeled_unos_valid)=preparar_dataset(data,target,fold=fold)
  embedding=data
  if reduccion==True:
    embedding=umap.UMAP(random_state=75).fit_transform(data)
  (puntos_ceros_umap,puntos_unos_umap,X_unlabeled_ceros_umap,X_unlabeled_unos_umap,X_unlabeled_ceros_valid_umap,X_unlabeled_unos_valid_umap)=preparar_dataset(embedding,target,fold=fold)


  bien=0
  mal=0
  dudoso=0

  puntos_ceros_final=puntos_ceros.tolist()
  puntos_unos_final=puntos_unos.tolist()
  puntos_dudosos_final=[]

  dgms_ceros = ripser(puntos_ceros_umap)['dgms'][dim]
  dgms_unos = ripser(puntos_unos_umap)['dgms'][dim]
  for i,punto in enumerate(X_unlabeled_unos_umap):
    clase=func(punto,puntos_ceros_umap,puntos_unos_umap,dgms_ceros,dgms_unos,th,dim)
    if clase==1:
      bien=bien+1
      puntos_unos_final.append(X_unlabeled_unos[i])
    elif clase==0:
      mal=mal+1
      puntos_ceros_final.append(X_unlabeled_unos[i])
    else:
      dudoso=dudoso+1
      puntos_dudosos_final.append(X_unlabeled_unos[i])



  for i,punto in enumerate(X_unlabeled_ceros_umap):
    clase=func(punto,puntos_ceros_umap,puntos_unos_umap,dgms_ceros,dgms_unos,th,dim)
    if clase==1:
      mal=mal+1
      puntos_unos_final.append(X_unlabeled_ceros[i])
    elif clase==0:
      bien=bien+1
      puntos_ceros_final.append(X_unlabeled_ceros[i])
    else:
      dudoso=dudoso+1
      puntos_dudosos_final.append(X_unlabeled_ceros[i])

  metrics_svm,metrics_rf=comprobar_accuracy(np.array(puntos_ceros_final),np.array(puntos_unos_final),X_unlabeled_ceros_valid,X_unlabeled_unos_valid)

  return (bien, mal,dudoso,np.array(puntos_ceros_final),np.array(puntos_unos_final),np.array(puntos_dudosos_final),metrics_svm,metrics_rf)

In [ ]:
def resumen(bien,mal,dudosos):
  print('-----------------RESUMEN------------------')
  print('Correct '+str(bien))
  print('Wrong '+str(mal))
  print('Unlabelled '+str(dudosos))
  print('')
  acc=0
  if bien+mal!=0:
    print('Correct percentage '+str(bien/(bien+mal)))
    acc=bien/(bien+mal)
  else:
    print('Correct percentage 0')
  print('Labelled percentage '+str((bien+mal)/(bien+mal+dudosos)))
  anotados=(bien+mal)/(bien+mal+dudosos)
  return acc,anotados

In [ ]:
import gudhi as gd

def obtener_radio(puntos):
  skeleton = gd.RipsComplex(points = puntos, max_edge_length = 90000000000)
  simplex_tree = skeleton.create_simplex_tree()
  filt=simplex_tree.get_filtration()
  *_, last0 = filt
  return last0[1]

In [ ]:
def analizar_punto_gudhi(punto,puntos_ceros,puntos_unos,last0,last1):
  last_0=obtener_radio(np.array([punto.tolist()]+puntos_ceros.tolist()))
  last_1=obtener_radio(np.array([punto.tolist()]+puntos_unos.tolist()))


  if abs(last0-last_0)<0.00001 and abs(last1-last_1)>0.00001:
    clase=0
  elif abs(last1-last_1)<0.00001 and abs(last0-last_0)>0.00001:
    clase=1
  else:
    clase=-1
  return clase

In [ ]:
def analizar_punto_gudhi2(punto,puntos_ceros,puntos_unos,last0,last1):

  last_0=obtener_radio(np.array([punto.tolist()]+puntos_ceros.tolist()))
  last_1=obtener_radio(np.array([punto.tolist()]+puntos_unos.tolist()))

  dif0=abs(last0-last_0)
  dif1=abs(last1-last_1)
  if dif0==0 and dif1==0:
    clase=-1
  elif dif0<dif1:
    clase=0
  else:
    clase=1
  return clase

In [ ]:
def analizar_puntos_gudhi(data,target,func,reduccion=False,fold=0):
  (puntos_ceros,puntos_unos,X_unlabeled_ceros,X_unlabeled_unos,X_unlabeled_ceros_valid,X_unlabeled_unos_valid)=preparar_dataset(data,target,fold=fold)
  embedding=data
  if reduccion==True:
    embedding=umap.UMAP(random_state=75).fit_transform(data)
  (puntos_ceros_umap,puntos_unos_umap,X_unlabeled_ceros_umap,X_unlabeled_unos_umap,X_unlabeled_ceros_valid_umap,X_unlabeled_unos_valid_umap)=preparar_dataset(embedding,target,fold=fold)


  bien=0
  mal=0
  dudoso=0

  puntos_ceros_final=puntos_ceros.tolist()
  puntos_unos_final=puntos_unos.tolist()
  puntos_dudosos_final=[]

  last0=obtener_radio(puntos_ceros_umap)
  last1=obtener_radio(puntos_unos_umap)
  for i,punto in enumerate(X_unlabeled_unos_umap):
    clase=func(punto,puntos_ceros_umap,puntos_unos_umap,last0,last1)
    if clase==1:
      bien=bien+1
      puntos_unos_final.append(X_unlabeled_unos[i])
    elif clase==0:
      mal=mal+1
      puntos_ceros_final.append(X_unlabeled_unos[i])
    else:
      dudoso=dudoso+1
      puntos_dudosos_final.append(X_unlabeled_unos[i])



  for i,punto in enumerate(X_unlabeled_ceros_umap):
    clase=func(punto,puntos_ceros_umap,puntos_unos_umap,last0,last1)
    if clase==1:
      mal=mal+1
      puntos_unos_final.append(X_unlabeled_ceros[i])
    elif clase==0:
      bien=bien+1
      puntos_ceros_final.append(X_unlabeled_ceros[i])
    else:
      dudoso=dudoso+1
      puntos_dudosos_final.append(X_unlabeled_ceros[i])

  metrics_svm,metrics_rf=comprobar_accuracy(np.array(puntos_ceros_final),np.array(puntos_unos_final),X_unlabeled_ceros_valid,X_unlabeled_unos_valid)

  return (bien, mal,dudoso,np.array(puntos_ceros_final),np.array(puntos_unos_final),np.array(puntos_dudosos_final),metrics_svm,metrics_rf)

In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score


def comprobar_accuracy(puntos_ceros_final,puntos_unos_final,X_unlabeled_ceros_valid,X_unlabeled_unos_valid):
    datos=np.concatenate((puntos_ceros_final,puntos_unos_final),axis=0)
    labels=np.concatenate((np.zeros(puntos_ceros_final.shape[0]),np.ones(puntos_unos_final.shape[0])))

    modelSVMLineal = SVC(kernel="linear",probability=True,random_state=90,max_iter=10000)
    modelSVMLineal.fit(datos, labels)


    predicted_ceros=np.array(modelSVMLineal.predict(X_unlabeled_ceros_valid))
    target_ceros=np.zeros_like(predicted_ceros)



    predicted_unos=np.array(modelSVMLineal.predict(X_unlabeled_unos_valid))
    target_unos=np.ones_like(predicted_unos)


    predicted=np.concatenate((predicted_ceros,predicted_unos))
    target=np.concatenate((target_ceros,target_unos))

    acc_svm=accuracy_score(target,predicted)
    pre_svm=precision_score(target,predicted)
    f1_svm=f1_score(target,predicted)
    rec_svm=recall_score(target,predicted)




    print("--------------Classifier SVM--------------")
    print("Accuracy: "+str(acc_svm))
    print("Precision: "+str(pre_svm))
    print("Recall: "+str(rec_svm))
    print("F1: "+str(f1_svm))
    print("")




    modelRandomForest = RandomForestClassifier(random_state=90)
    modelRandomForest.fit(datos, labels)




    predicted_ceros=np.array(modelRandomForest.predict(X_unlabeled_ceros_valid))
    target_ceros=np.zeros_like(predicted_ceros)


    predicted_unos=np.array(modelRandomForest.predict(X_unlabeled_unos_valid))
    target_unos=np.ones_like(predicted_unos)




    predicted=np.concatenate((predicted_ceros,predicted_unos))
    target=np.concatenate((target_ceros,target_unos))

    acc_rf=accuracy_score(target,predicted)
    pre_rf=precision_score(target,predicted)
    f1_rf=f1_score(target,predicted)
    rec_rf=recall_score(target,predicted)

    print("-------------- Classifier Random Forest--------------")
    print("Accuracy: "+str(acc_svm))
    print("Precision: "+str(pre_svm))
    print("Recall: "+str(rec_svm))
    print("F1: "+str(f1_svm))
    print("")
    return (acc_svm,pre_svm,rec_svm,f1_svm),(acc_rf,pre_rf,rec_rf,f1_rf)

In [ ]:
def label_prop(data,target,fold=0,th=0):
  (puntos_ceros,puntos_unos,X_unlabeled_ceros,X_unlabeled_unos,puntos_ceros_valid,puntos_unos_valid)=preparar_dataset(data,target,fold=fold)
  datos=np.concatenate((puntos_ceros,puntos_unos),axis=0)
  labels=np.concatenate((np.zeros(puntos_ceros.shape[0]),np.ones(puntos_unos.shape[0])))

  label_prop_model = LabelPropagation(kernel='knn')
  label_prop_model.fit(datos, labels)

###################ZEROS#################
  pred_ceros=label_prop_model.predict_proba(X_unlabeled_ceros)

  prob_ceros=np.max(pred_ceros,axis=1)
  clases_ceros=np.argmax(pred_ceros,axis=1)

  ind_dudosos_ceros=np.where(prob_ceros<th)[0]
  ind_no_dudosos_ceros=np.where(prob_ceros>=th)[0]


  unos=np.where(clases_ceros==1)[0]

  ceros=np.where(clases_ceros==0)[0]


  unos=unos[np.isin(unos,ind_no_dudosos_ceros)]
  mal=unos.shape[0]
  ceros=ceros[np.isin(ceros,ind_no_dudosos_ceros)]
  bien=ceros.shape[0]


  datos_uno=np.array(X_unlabeled_ceros)[unos]
  datos_cero=np.array(X_unlabeled_ceros)[ceros]

  nuevos_puntos_cero=np.asarray(list(puntos_ceros)+list(datos_cero))
  nuevos_puntos_uno=np.asarray(list(puntos_unos)+list(datos_uno))

  dudoso_cero=np.array(X_unlabeled_ceros)[ind_dudosos_ceros]
  dudoso=ind_dudosos_ceros.shape[0]


######################ONES#############################################
  pred_unos=label_prop_model.predict_proba(X_unlabeled_unos)

  prob_unos=np.max(pred_unos,axis=1)
  clases_unos=np.argmax(pred_unos,axis=1)

  ind_dudosos_unos=np.where(prob_unos<th)[0]
  ind_no_dudosos_unos=np.where(prob_unos>=th)[0]


  unos=np.where(clases_unos==1)[0]

  ceros=np.where(clases_unos==0)[0]


  unos=unos[np.isin(unos,ind_no_dudosos_unos)]
  bien+=unos.shape[0]
  ceros=ceros[np.isin(ceros,ind_no_dudosos_unos)]
  mal+=ceros.shape[0]


  datos_uno=np.array(X_unlabeled_unos)[unos]
  datos_cero=np.array(X_unlabeled_unos)[ceros]


  nuevos_puntos_cero=np.asarray(list(nuevos_puntos_cero)+list(datos_cero))
  nuevos_puntos_uno=np.asarray(list(nuevos_puntos_uno)+list(datos_uno))

  dudoso_uno=np.array(X_unlabeled_unos)[ind_dudosos_unos]

  dudosos=np.concatenate((dudoso_cero,dudoso_uno))
  dudoso+=ind_dudosos_unos.shape[0]

  metrics_svm,metrics_rf=comprobar_accuracy(nuevos_puntos_cero,nuevos_puntos_uno,puntos_ceros_valid,puntos_unos_valid)

  return (bien, mal,dudoso, nuevos_puntos_cero, nuevos_puntos_uno, dudoso,metrics_svm,metrics_rf)

In [ ]:
def label_spread(data,target,fold=0,th=0):
  (puntos_ceros,puntos_unos,X_unlabeled_ceros,X_unlabeled_unos,puntos_ceros_valid,puntos_unos_valid)=preparar_dataset(data,target,fold=fold)
  datos=np.concatenate((puntos_ceros,puntos_unos),axis=0)
  labels=np.concatenate((np.zeros(puntos_ceros.shape[0]),np.ones(puntos_unos.shape[0])))

  label_prop_model = LabelSpreading(kernel='knn')
  label_prop_model.fit(datos, labels)

###################ZEROS#################
  pred_ceros=label_prop_model.predict_proba(X_unlabeled_ceros)

  prob_ceros=np.max(pred_ceros,axis=1)
  clases_ceros=np.argmax(pred_ceros,axis=1)

  ind_dudosos_ceros=np.where(prob_ceros<th)[0]
  ind_no_dudosos_ceros=np.where(prob_ceros>=th)[0]


  unos=np.where(clases_ceros==1)[0]

  ceros=np.where(clases_ceros==0)[0]


  unos=unos[np.isin(unos,ind_no_dudosos_ceros)]
  mal=unos.shape[0]
  ceros=ceros[np.isin(ceros,ind_no_dudosos_ceros)]
  bien=ceros.shape[0]


  datos_uno=np.array(X_unlabeled_ceros)[unos]
  datos_cero=np.array(X_unlabeled_ceros)[ceros]

  nuevos_puntos_cero=np.asarray(list(puntos_ceros)+list(datos_cero))
  nuevos_puntos_uno=np.asarray(list(puntos_unos)+list(datos_uno))

  dudoso_cero=np.array(X_unlabeled_ceros)[ind_dudosos_ceros]
  dudoso=ind_dudosos_ceros.shape[0]


######################ONES#############################################
  pred_unos=label_prop_model.predict_proba(X_unlabeled_unos)

  prob_unos=np.max(pred_unos,axis=1)
  clases_unos=np.argmax(pred_unos,axis=1)

  ind_dudosos_unos=np.where(prob_unos<th)[0]
  ind_no_dudosos_unos=np.where(prob_unos>=th)[0]


  unos=np.where(clases_unos==1)[0]

  ceros=np.where(clases_unos==0)[0]


  unos=unos[np.isin(unos,ind_no_dudosos_unos)]
  bien+=unos.shape[0]
  ceros=ceros[np.isin(ceros,ind_no_dudosos_unos)]
  mal+=ceros.shape[0]


  datos_uno=np.array(X_unlabeled_unos)[unos]
  datos_cero=np.array(X_unlabeled_unos)[ceros]


  nuevos_puntos_cero=np.asarray(list(nuevos_puntos_cero)+list(datos_cero))
  nuevos_puntos_uno=np.asarray(list(nuevos_puntos_uno)+list(datos_uno))

  dudoso_uno=np.array(X_unlabeled_unos)[ind_dudosos_unos]

  dudosos=np.concatenate((dudoso_cero,dudoso_uno))
  dudoso+=ind_dudosos_unos.shape[0]

  metrics_svm,metrics_rf=comprobar_accuracy(nuevos_puntos_cero,nuevos_puntos_uno,puntos_ceros_valid,puntos_unos_valid)

  return (bien, mal,dudoso, nuevos_puntos_cero, nuevos_puntos_uno, dudoso,metrics_svm,metrics_rf)

In [ ]:
def self_train(data,target,th=0,fold=0):
  (puntos_ceros,puntos_unos,X_unlabeled_ceros,X_unlabeled_unos,puntos_ceros_valid,puntos_unos_valid)=preparar_dataset(data,target,fold=fold)
  datos=np.concatenate((puntos_ceros,puntos_unos),axis=0)
  labels=np.concatenate((np.zeros(puntos_ceros.shape[0]),np.ones(puntos_unos.shape[0])))

  svc = SVC(probability=True, gamma="auto")
  self_training_model = SelfTrainingClassifier(svc,threshold=th)
  self_training_model.fit(datos, labels)
###################ZEROS#################
  pred_ceros=self_training_model.predict_proba(X_unlabeled_ceros)

  prob_ceros=np.max(pred_ceros,axis=1)
  clases_ceros=np.argmax(pred_ceros,axis=1)

  ind_dudosos_ceros=np.where(prob_ceros<th)[0]
  ind_no_dudosos_ceros=np.where(prob_ceros>=th)[0]


  unos=np.where(clases_ceros==1)[0]

  ceros=np.where(clases_ceros==0)[0]


  unos=unos[np.isin(unos,ind_no_dudosos_ceros)]
  mal=unos.shape[0]
  ceros=ceros[np.isin(ceros,ind_no_dudosos_ceros)]
  bien=ceros.shape[0]


  datos_uno=np.array(X_unlabeled_ceros)[unos]
  datos_cero=np.array(X_unlabeled_ceros)[ceros]

  nuevos_puntos_cero=np.asarray(list(puntos_ceros)+list(datos_cero))
  nuevos_puntos_uno=np.asarray(list(puntos_unos)+list(datos_uno))

  dudoso_cero=np.array(X_unlabeled_ceros)[ind_dudosos_ceros]
  dudoso=ind_dudosos_ceros.shape[0]


######################ONES#############################################
  pred_unos=self_training_model.predict_proba(X_unlabeled_unos)

  prob_unos=np.max(pred_unos,axis=1)
  clases_unos=np.argmax(pred_unos,axis=1)

  ind_dudosos_unos=np.where(prob_unos<th)[0]
  ind_no_dudosos_unos=np.where(prob_unos>=th)[0]


  unos=np.where(clases_unos==1)[0]

  ceros=np.where(clases_unos==0)[0]


  unos=unos[np.isin(unos,ind_no_dudosos_unos)]
  bien+=unos.shape[0]
  ceros=ceros[np.isin(ceros,ind_no_dudosos_unos)]
  mal+=ceros.shape[0]


  datos_uno=np.array(X_unlabeled_unos)[unos]
  datos_cero=np.array(X_unlabeled_unos)[ceros]


  nuevos_puntos_cero=np.asarray(list(nuevos_puntos_cero)+list(datos_cero))
  nuevos_puntos_uno=np.asarray(list(nuevos_puntos_uno)+list(datos_uno))

  dudoso_uno=np.array(X_unlabeled_unos)[ind_dudosos_unos]

  dudosos=np.concatenate((dudoso_cero,dudoso_uno))
  dudoso+=ind_dudosos_unos.shape[0]

  metrics_svm,metrics_rf=comprobar_accuracy(nuevos_puntos_cero,nuevos_puntos_uno,puntos_ceros_valid,puntos_unos_valid)

  return (bien, mal,dudoso, nuevos_puntos_cero, nuevos_puntos_uno, dudoso,metrics_svm,metrics_rf)

In [ ]:
with open(path+'PrimaIndian.csv', 'w', newline='') as csvfile:
  spamwriter = csv.writer(csvfile)
  spamwriter.writerow(['Method','acc_lab_1','%lab_1','svm_1_acc','svm_1_pre','svm_1_rec','svm_1_f1','rf_1_acc','rf_1_pre','rf_1_rec','rf_1_f1','acc_lab_2','%lab_2','svm_2_acc','svm_2_pre','svm_2_rec','svm_2_f1','rf_2_acc','rf_2_pre','rf_2_rec','rf_2_f1','acc_lab_3','%lab_3','svm_3_acc','svm_3_pre','svm_3_rec','svm_3_f1','rf_3_acc','rf_3_pre','rf_3_rec','rf_3_f1','acc_lab_4','%lab_4','svm_4_acc','svm_4_pre','svm_4_rec','svm_4_f1','rf_4_acc','rf_4_pre','rf_4_rec','rf_4_f1','acc_lab_5','%lab_5','svm_5_acc','svm_5_pre','svm_5_rec','svm_5_f1','rf_5_acc','rf_5_pre','rf_5_rec','rf_5_f1'])

# Bottleneck

In [ ]:
k=5

In [ ]:
fila=['Bottleneck_00']
for i in range(k):
  (data,target) = load_prima()
  (bien,mal,dudoso,pcero,puno,pdudoso,acc_svm,acc_rf)=analizar_puntos(data,target,analizar_punto,fold=i)
  acc,anotados=resumen(bien,mal,dudoso)
  print("")
  print("")
  fila.append(acc)
  fila.append(anotados)
  fila.append(acc_svm[0])
  fila.append(acc_svm[1])
  fila.append(acc_svm[2])
  fila.append(acc_svm[3])
  fila.append(acc_rf[0])
  fila.append(acc_rf[1])
  fila.append(acc_rf[2])
  fila.append(acc_rf[3])
with open(path+'PrimaIndian.csv', 'a', newline='') as csvfile:
  spamwriter = csv.writer(csvfile)
  spamwriter.writerow(fila)

# Bottleneck threshold

In [ ]:
fila=['Bottleneck_08']
for i in range(k):
  (data,target) = load_prima()
  (bien,mal,dudoso,pcero,puno,pdudoso,acc_svm,acc_rf)=analizar_puntos(data,target,analizar_punto_th,0.8,fold=i)
  acc,anotados=resumen(bien,mal,dudoso)
  print("")
  print("")
  fila.append(acc)
  fila.append(anotados)
  fila.append(acc_svm[0])
  fila.append(acc_svm[1])
  fila.append(acc_svm[2])
  fila.append(acc_svm[3])
  fila.append(acc_rf[0])
  fila.append(acc_rf[1])
  fila.append(acc_rf[2])
  fila.append(acc_rf[3])
with open(path+'PrimaIndian.csv', 'a', newline='') as csvfile:
  spamwriter = csv.writer(csvfile)
  spamwriter.writerow(fila)

# Bottleneck threshold 0.6

In [ ]:
fila=['Bottleneck_06']
for i in range(k):
  (data,target) = load_prima()
  (bien,mal,dudoso,pcero,puno,pdudoso,acc_svm,acc_rf)=analizar_puntos(data,target,analizar_punto_th,0.6,fold=i)
  acc,anotados=resumen(bien,mal,dudoso)
  print("")
  print("")
  fila.append(acc)
  fila.append(anotados)
  fila.append(acc_svm[0])
  fila.append(acc_svm[1])
  fila.append(acc_svm[2])
  fila.append(acc_svm[3])
  fila.append(acc_rf[0])
  fila.append(acc_rf[1])
  fila.append(acc_rf[2])
  fila.append(acc_rf[3])
with open(path+'PrimaIndian.csv', 'a', newline='') as csvfile:
  spamwriter = csv.writer(csvfile)
  spamwriter.writerow(fila)

# Wasserstein

In [ ]:
fila=['Wasserstein_00']
for i in range(k):
  (data,target) = load_prima()
  (bien,mal,dudoso,pcero,puno,pdudoso,acc_svm,acc_rf)=analizar_puntos(data,target,analizar_punto_Wasserstein,fold=i)
  acc,anotados=resumen(bien,mal,dudoso)
  print("")
  print("")
  fila.append(acc)
  fila.append(anotados)
  fila.append(acc_svm[0])
  fila.append(acc_svm[1])
  fila.append(acc_svm[2])
  fila.append(acc_svm[3])
  fila.append(acc_rf[0])
  fila.append(acc_rf[1])
  fila.append(acc_rf[2])
  fila.append(acc_rf[3])
with open(path+'PrimaIndian.csv', 'a', newline='') as csvfile:
  spamwriter = csv.writer(csvfile)
  spamwriter.writerow(fila)

# Wasserstein threshold

In [ ]:
fila=['Wasserstein_08']
for i in range(k):
  (data,target) = load_prima()
  (bien,mal,dudoso,pcero,puno,pdudoso,acc_svm,acc_rf)=analizar_puntos(data,target,analizar_punto_Wasserstein_th,0.8,fold=i)
  acc,anotados=resumen(bien,mal,dudoso)
  print("")
  print("")
  fila.append(acc)
  fila.append(anotados)
  fila.append(acc_svm[0])
  fila.append(acc_svm[1])
  fila.append(acc_svm[2])
  fila.append(acc_svm[3])
  fila.append(acc_rf[0])
  fila.append(acc_rf[1])
  fila.append(acc_rf[2])
  fila.append(acc_rf[3])
with open(path+'PrimaIndian.csv', 'a', newline='') as csvfile:
  spamwriter = csv.writer(csvfile)
  spamwriter.writerow(fila)

# Wasserstein threshold 0.6

In [ ]:
fila=['Wasserstein_06']
for i in range(k):
  (data,target) = load_prima()
  (bien,mal,dudoso,pcero,puno,pdudoso,acc_svm,acc_rf)=analizar_puntos(data,target,analizar_punto_Wasserstein_th,0.6,fold=i)
  acc,anotados=resumen(bien,mal,dudoso)
  print("")
  print("")
  fila.append(acc)
  fila.append(anotados)
  fila.append(acc_svm[0])
  fila.append(acc_svm[1])
  fila.append(acc_svm[2])
  fila.append(acc_svm[3])
  fila.append(acc_rf[0])
  fila.append(acc_rf[1])
  fila.append(acc_rf[2])
  fila.append(acc_rf[3])
with open(path+'PrimaIndian.csv', 'a', newline='') as csvfile:
  spamwriter = csv.writer(csvfile)
  spamwriter.writerow(fila)

# Gudhi

In [ ]:
fila=['Gudhi']
for i in range(k):
  (data,target) = load_prima()
  (bien,mal,dudoso,pcero,puno,pdudoso,acc_svm,acc_rf)=analizar_puntos_gudhi(data,target,analizar_punto_gudhi,fold=i)
  acc,anotados=resumen(bien,mal,dudoso)
  print("")
  print("")
  fila.append(acc)
  fila.append(anotados)
  fila.append(acc_svm[0])
  fila.append(acc_svm[1])
  fila.append(acc_svm[2])
  fila.append(acc_svm[3])
  fila.append(acc_rf[0])
  fila.append(acc_rf[1])
  fila.append(acc_rf[2])
  fila.append(acc_rf[3])
with open(path+'PrimaIndian.csv', 'a', newline='') as csvfile:
  spamwriter = csv.writer(csvfile)
  spamwriter.writerow(fila)

# Gudhi 2

In [ ]:
fila=['Gudhi2']
for i in range(k):
  (data,target) = load_prima()
  (bien,mal,dudoso,pcero,puno,pdudoso,acc_svm,acc_rf)=analizar_puntos_gudhi(data,target,analizar_punto_gudhi2,fold=i)
  acc,anotados=resumen(bien,mal,dudoso)
  print("")
  print("")
  fila.append(acc)
  fila.append(anotados)
  fila.append(acc_svm[0])
  fila.append(acc_svm[1])
  fila.append(acc_svm[2])
  fila.append(acc_svm[3])
  fila.append(acc_rf[0])
  fila.append(acc_rf[1])
  fila.append(acc_rf[2])
  fila.append(acc_rf[3])
with open(path+'PrimaIndian.csv', 'a', newline='') as csvfile:
  spamwriter = csv.writer(csvfile)
  spamwriter.writerow(fila)

# LabelPropagation

In [ ]:
fila=['LabelPropagation']
for i in range(k):
  (data,target) = load_prima()
  (bien,mal,dudoso,pcero,puno,pdudoso,acc_svm,acc_rf)=label_prop(data,target,fold=i)
  acc,anotados=resumen(bien,mal,dudoso)
  print("")
  print("")
  fila.append(acc)
  fila.append(anotados)
  fila.append(acc_svm[0])
  fila.append(acc_svm[1])
  fila.append(acc_svm[2])
  fila.append(acc_svm[3])
  fila.append(acc_rf[0])
  fila.append(acc_rf[1])
  fila.append(acc_rf[2])
  fila.append(acc_rf[3])
with open(path+'PrimaIndian.csv', 'a', newline='') as csvfile:
  spamwriter = csv.writer(csvfile)
  spamwriter.writerow(fila)

# LabelPropagation 0.8

In [ ]:
fila=['LabelPropagation_08']
for i in range(k):
  (data,target) = load_prima()
  (bien,mal,dudoso,pcero,puno,pdudoso,acc_svm,acc_rf)=label_prop(data,target,fold=i,th=0.8)
  acc,anotados=resumen(bien,mal,dudoso)
  print("")
  print("")
  fila.append(acc)
  fila.append(anotados)
  fila.append(acc_svm[0])
  fila.append(acc_svm[1])
  fila.append(acc_svm[2])
  fila.append(acc_svm[3])
  fila.append(acc_rf[0])
  fila.append(acc_rf[1])
  fila.append(acc_rf[2])
  fila.append(acc_rf[3])
with open(path+'PrimaIndian.csv', 'a', newline='') as csvfile:
  spamwriter = csv.writer(csvfile)
  spamwriter.writerow(fila)

# LabelPropagation 0.6

In [ ]:
fila=['LabelPropagation_06']
for i in range(k):
  (data,target) = load_prima()
  (bien,mal,dudoso,pcero,puno,pdudoso,acc_svm,acc_rf)=label_prop(data,target,fold=i,th=0.6)
  acc,anotados=resumen(bien,mal,dudoso)
  print("")
  print("")
  fila.append(acc)
  fila.append(anotados)
  fila.append(acc_svm[0])
  fila.append(acc_svm[1])
  fila.append(acc_svm[2])
  fila.append(acc_svm[3])
  fila.append(acc_rf[0])
  fila.append(acc_rf[1])
  fila.append(acc_rf[2])
  fila.append(acc_rf[3])
with open(path+'PrimaIndian.csv', 'a', newline='') as csvfile:
  spamwriter = csv.writer(csvfile)
  spamwriter.writerow(fila)

# LabelSpreading

In [ ]:
fila=['LabelSpreading']
for i in range(k):
  (data,target) = load_prima()
  (bien,mal,dudoso,pcero,puno,pdudoso,acc_svm,acc_rf)=label_spread(data,target,fold=i)
  acc,anotados=resumen(bien,mal,dudoso)
  print("")
  print("")
  fila.append(acc)
  fila.append(anotados)
  fila.append(acc_svm[0])
  fila.append(acc_svm[1])
  fila.append(acc_svm[2])
  fila.append(acc_svm[3])
  fila.append(acc_rf[0])
  fila.append(acc_rf[1])
  fila.append(acc_rf[2])
  fila.append(acc_rf[3])
with open(path+'PrimaIndian.csv', 'a', newline='') as csvfile:
  spamwriter = csv.writer(csvfile)
  spamwriter.writerow(fila)

# LabelSpreading 0.8

In [ ]:
fila=['LabelSpreading_08']
for i in range(k):
  (data,target) = load_prima()
  (bien,mal,dudoso,pcero,puno,pdudoso,acc_svm,acc_rf)=label_spread(data,target,fold=i,th=0.8)
  acc,anotados=resumen(bien,mal,dudoso)
  print("")
  print("")
  fila.append(acc)
  fila.append(anotados)
  fila.append(acc_svm[0])
  fila.append(acc_svm[1])
  fila.append(acc_svm[2])
  fila.append(acc_svm[3])
  fila.append(acc_rf[0])
  fila.append(acc_rf[1])
  fila.append(acc_rf[2])
  fila.append(acc_rf[3])
with open(path+'PrimaIndian.csv', 'a', newline='') as csvfile:
  spamwriter = csv.writer(csvfile)
  spamwriter.writerow(fila)

# LabelSpreading 0.6

In [ ]:
fila=['LabelSpreading_06']
for i in range(k):
  (data,target) = load_prima()
  (bien,mal,dudoso,pcero,puno,pdudoso,acc_svm,acc_rf)=label_spread(data,target,fold=i,th=0.6)
  acc,anotados=resumen(bien,mal,dudoso)
  print("")
  print("")
  fila.append(acc)
  fila.append(anotados)
  fila.append(acc_svm[0])
  fila.append(acc_svm[1])
  fila.append(acc_svm[2])
  fila.append(acc_svm[3])
  fila.append(acc_rf[0])
  fila.append(acc_rf[1])
  fila.append(acc_rf[2])
  fila.append(acc_rf[3])
with open(path+'PrimaIndian.csv', 'a', newline='') as csvfile:
  spamwriter = csv.writer(csvfile)
  spamwriter.writerow(fila)

# SelfTraining

In [ ]:
fila=['SelfTraining_00']
for i in range(k):
  (data,target) = load_prima()
  (bien,mal,dudoso,pcero,puno,pdudoso,acc_svm,acc_rf)=self_train(data,target,th=0,fold=i)
  acc,anotados=resumen(bien,mal,dudoso)
  print("")
  print("")
  fila.append(acc)
  fila.append(anotados)
  fila.append(acc_svm[0])
  fila.append(acc_svm[1])
  fila.append(acc_svm[2])
  fila.append(acc_svm[3])
  fila.append(acc_rf[0])
  fila.append(acc_rf[1])
  fila.append(acc_rf[2])
  fila.append(acc_rf[3])
with open(path+'PrimaIndian.csv', 'a', newline='') as csvfile:
  spamwriter = csv.writer(csvfile)
  spamwriter.writerow(fila)

# SelfTraining 0.8

In [ ]:
fila=['SelfTraining_08']
for i in range(k):
  (data,target) = load_prima()
  (bien,mal,dudoso,pcero,puno,pdudoso,acc_svm,acc_rf)=self_train(data,target,th=0.8,fold=i)
  acc,anotados=resumen(bien,mal,dudoso)
  print("")
  print("")
  fila.append(acc)
  fila.append(anotados)
  fila.append(acc_svm[0])
  fila.append(acc_svm[1])
  fila.append(acc_svm[2])
  fila.append(acc_svm[3])
  fila.append(acc_rf[0])
  fila.append(acc_rf[1])
  fila.append(acc_rf[2])
  fila.append(acc_rf[3])
with open(path+'PrimaIndian.csv', 'a', newline='') as csvfile:
  spamwriter = csv.writer(csvfile)
  spamwriter.writerow(fila)

# SelfTraining 0.6

In [ ]:
fila=['SelfTraining_06']
for i in range(k):
  (data,target) = load_prima()
  (bien,mal,dudoso,pcero,puno,pdudoso,acc_svm,acc_rf)=self_train(data,target,th=0.6,fold=i)
  acc,anotados=resumen(bien,mal,dudoso)
  print("")
  print("")
  fila.append(acc)
  fila.append(anotados)
  fila.append(acc_svm[0])
  fila.append(acc_svm[1])
  fila.append(acc_svm[2])
  fila.append(acc_svm[3])
  fila.append(acc_rf[0])
  fila.append(acc_rf[1])
  fila.append(acc_rf[2])
  fila.append(acc_rf[3])
with open(path+'PrimaIndian.csv', 'a', newline='') as csvfile:
  spamwriter = csv.writer(csvfile)
  spamwriter.writerow(fila)

# Base

In [ ]:
fila=['Base']
for i in range(k):
  (data,target) = load_prima()
  (puntos_ceros,puntos_unos,X_unlabeled_ceros,X_unlabeled_unos,puntos_ceros_valid,puntos_unos_valid)=preparar_dataset(data,target,fold=i)
  acc_svm,acc_rf=comprobar_accuracy(puntos_ceros,puntos_unos,puntos_ceros_valid,puntos_unos_valid)
  print("")
  print("")
  fila.append(0)
  fila.append(0)
  fila.append(acc_svm[0])
  fila.append(acc_svm[1])
  fila.append(acc_svm[2])
  fila.append(acc_svm[3])
  fila.append(acc_rf[0])
  fila.append(acc_rf[1])
  fila.append(acc_rf[2])
  fila.append(acc_rf[3])
with open(path+'PrimaIndian.csv', 'a', newline='') as csvfile:
  spamwriter = csv.writer(csvfile)
  spamwriter.writerow(fila)

# Todos

In [ ]:
k=5

In [ ]:
fila=['Full']
for i in range(k):
  (data,target) = load_prima()
  (puntos_ceros,puntos_unos,X_unlabeled_ceros,X_unlabeled_unos,puntos_ceros_valid,puntos_unos_valid)=preparar_dataset(data,target,fold=i)
  acc_svm,acc_rf=comprobar_accuracy(np.concatenate((puntos_ceros,X_unlabeled_ceros)),np.concatenate((puntos_unos,X_unlabeled_unos)),puntos_ceros_valid,puntos_unos_valid)
  print("")
  print("")
  fila.append(0)
  fila.append(0)
  fila.append(acc_svm[0])
  fila.append(acc_svm[1])
  fila.append(acc_svm[2])
  fila.append(acc_svm[3])
  fila.append(acc_rf[0])
  fila.append(acc_rf[1])
  fila.append(acc_rf[2])
  fila.append(acc_rf[3])
with open(path+'PrimaIndian.csv', 'a', newline='') as csvfile:
  spamwriter = csv.writer(csvfile)
  spamwriter.writerow(fila)